Import the required libraries.

In [0]:
import math
import numpy as np
import random
import pandas as pd

**Define the objective function**:
   
   Himmelblau function: It is a continuous and non-convex multi-modal function.

Min. f(x,y) = (x2 + y -11) 2 + (x +y2 -7)2

Subjected to the constraints of:

26 - (x-5)2 - y2 ≥ 0

20 - 4x - y ≥ 0

x ε [-5, 5]; y ε [-5, 5]

In [0]:
def myobj(p1):
    F=[]
    for i in range (len(p1)):
        x=p1.loc[i]
        f=(((x[0]**2)+x[1]-11)**2)+((x[0]+x[1]**2)-7)**2
        F.append(f)
    return F

**Search space and termination criteria**


In [0]:
pop_size = 25
Gen = 1000
lb=[-5,-5]
ub=[5,5]

**Initial population**

That is, generating random solutions, within the given range of variables. These random solutions serve as the starting point for our algorithm.

In [0]:
def initialpopulation(mini,maxi,pop_size):
    pop=[]        
    for i in range(pop_size):
        p=[]        
        for a,b in zip(mini,maxi):
            p.append(a + (b-a) * random.random())
        pop.append(p)    
    ini_pop=pd.DataFrame(pop)        
    return ini_pop

**Update population**

In every iteration, the solutions are updated based on the strategy.

In [0]:
def updatepopulation(p1,dim):      
    best_x=np.array(p1.loc[p1['f'].idxmin][0:dim])    
    worst_x=np.array(p1.loc[p1['f'].idxmax][0:dim])
    new_x=[]
    for i in range(len(p1)):
        old_x=np.array(p1.loc[i][0:dim])           
        r1=np.random.random(dim)
        #r2=np.random.random(dim)
        #new_x.append(old_x+r1*(best_x-abs(old_x))-r2*(worst_x-abs(old_x)))
        new_x.append(old_x+r1*(best_x-worst_x))    
    new_p1=pd.DataFrame(new_x)    
    return new_p1

**Greedy selection**

Inherently, in every generation the algorithm keeps only the good solution and completely discards the poor or inferior solution

In [0]:
def greedyselector(p1,new_p1):    
    for i in range(len(p1)):        
        if p1.loc[i]['f']>new_p1.loc[i]['f']:                 
            p1.loc[i]=new_p1.loc[i]    
    return p1

**Trimming**

It is imperative that the combination of variables suggested by the algorithm algorithm must lie within the prescribed bounds. This can be achieved by a number of different approaches. One approach is to trim the variables to their respective bounds.

In [0]:
def trimr(new_p1,lb,ub):    
    col=new_p1.columns.values    
    for i in range(len(new_p1)):        
        for j in range(len(col)):            
            if new_p1.loc[i][j]>ub[j]:                
                  new_p1.loc[i][j]=ub[j]            
            elif new_p1.loc[i][j]<lb[j]:                          
                  new_p1.loc[i][j]=lb[j]    
    return new_p1

**Looping**

All the above mentioned functions have to be executed in a loop, in order to perform iterations.

In [0]:
def Rao1(*argv):
    pop_size, Gen, mini, maxi= argv
    lb=np.array(mini)
    ub=np.array(maxi)
    p1=initialpopulation(lb,ub,pop_size)
    p1['f']=myobj(p1)
    
    dim=len(lb)
    gen=0
    best=[]
    while (gen<Gen):
        new_p1=updatepopulation(p1,dim)
        new_p1=trimr(new_p1,lb,ub)
        new_p1['f']=myobj(new_p1)
        p1=greedyselector(p1,new_p1)
        gen=gen+1
        #print(gen)
        best=p1['f'].min()
        xbest=p1.loc[p1['f'].idxmin()][0:dim].tolist() 
      #  print('Best={}'.format(best))
       # print('xbest={}'.format(xbest))
    return best,xbest

**Executing the code**

Import all the above mentioned functions. And then the algorithm can be executed as follows:

In [0]:
best,xbest = Rao1(pop_size, Gen, lb, ub)
print('The objective function value = {}'.format(best))
print('The optimum values of variables = {}'.format(xbest))